<a href="https://colab.research.google.com/github/NaotoKubota/Seimei-2021/blob/master/2021%E5%B9%B4%E5%BA%A6%E7%94%9F%E5%91%BD%E7%A7%91%E5%AD%A6%E7%A7%91%E8%AC%9B%E7%BE%A9%E7%94%A8%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%97%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2021年度生命科学科講義用スクリプト v1.1.0

1. Minicondaによるツールのインストール（seqtk, fastp, hisat2, samtools, subread）
2. RNA-seqデータのダウンロード
3. ダウンサンプリング（seqtk）
4. クオリティーコントロール（fastp）
5. マッピング（hisat2） & samファイルをbamファイルに変換（samtools）
6. 発現量のカウント（featureCounts）

2021.07.14 久保田

## 1. Minicondaによるツールのインストール（seqtk, fastp, hisat2, samtools, subread）

In [ ]:
%%bash
# Minicondaのダウンロード
wget https://repo.continuum.io/miniconda/Miniconda3-py38_4.8.3-Linux-x86_64.sh && \
bash Miniconda3-py38_4.8.3-Linux-x86_64.sh -bfp /usr/local

In [ ]:
%%bash
# 必要なツールのインストール
conda config --add channels conda-forge
conda config --add channels bioconda
conda install seqtk fastp hisat2 samtools subread 

## 2. RNA-seqデータのダウンロード

In [ ]:
%%bash
# RNA-seqデータのダウンロード
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR650/003/SRR6503273/SRR6503273_1.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR650/003/SRR6503273/SRR6503273_2.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR650/004/SRR6503274/SRR6503274_1.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR650/004/SRR6503274/SRR6503274_2.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR650/005/SRR6503275/SRR6503275_1.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR650/005/SRR6503275/SRR6503275_2.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR650/006/SRR6503276/SRR6503276_1.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR650/006/SRR6503276/SRR6503276_2.fastq.gz

## 3. ダウンサンプリング（seqtk）

In [ ]:
%%bash

for SRR_ID in SRR650327{3..6};
do

  # 1000万リードをランダムサンプリング
  seqtk sample -s 100 ${SRR_ID}_1.fastq.gz 10000000 | gzip -c > ${SRR_ID}_1_10M.fastq.gz
  seqtk sample -s 100 ${SRR_ID}_2.fastq.gz 10000000 | gzip -c > ${SRR_ID}_2_10M.fastq.gz

  # 不要なファイルを削除
  rm ${SRR_ID}_1.fastq.gz ${SRR_ID}_2.fastq.gz

done

## 4. クオリティーコントロール（fastp）

In [ ]:
%%bash

for SRR_ID in SRR650327{3..6};
do

  # fastpを使ってクオリティーコントロール
  # tail末端の1塩基をトリミングする。 20bp以下は捨てる。2スレッド使う。
  fastp -i ${SRR_ID}_1_10M.fastq.gz -I ${SRR_ID}_2_10M.fastq.gz \
  -o ${SRR_ID}_1_10M_filter.fastq.gz -O ${SRR_ID}_2_10M_filter.fastq.gz \
  -h ${SRR_ID}_report.html -j ${SRR_ID}_report.json -q 15 -n 10 -t 1 -T 1 -l 20 -w 2

  # 元のfastqファイルを削除
  rm ${SRR_ID}_1_10M.fastq.gz ${SRR_ID}_2_10M.fastq.gz

done

## 5. マッピング（hisat2） & samファイルをbamファイルに変換（samtools）

In [ ]:
%%bash
# hisat2のindexをダウンロード
wget ftp://ftp.ccb.jhu.edu/pub/infphilo/hisat2/data/hg38.tar.gz
tar -zxvf hg38.tar.gz
rm hg38.tar.gz 

In [ ]:
%%bash

for SRR_ID in SRR650327{3..6};
do

  # hisat2によるマッピング
  hisat2 -t -p 2 -x hg38/genome \
  -1 ${SRR_ID}_1_10M_filter.fastq.gz -2 ${SRR_ID}_2_10M_filter.fastq.gz \
  -S ${SRR_ID}.sam --summary-file ${SRR_ID}.txt

  # samtoolsでsamをbamに変換 & sort
  samtools sort -@ 2 -O bam -o ${SRR_ID}.sort.bam ${SRR_ID}.sam

  # 不要なファイルの削除
  rm ${SRR_ID}_1_10M_filter.fastq.gz ${SRR_ID}_2_10M_filter.fastq.gz ${SRR_ID}.sam

done

## 6. 発現量のカウント（featureCounts）

In [ ]:
%%bash
# GTFファイルのダウンロード
wget ftp://ftp.ensembl.org/pub/release-98/gtf/homo_sapiens/Homo_sapiens.GRCh38.98.gtf.gz
gzip -d Homo_sapiens.GRCh38.98.gtf.gz

In [ ]:
%%bash
# featureCountsで発現量をカウント
featureCounts -p -B -t exon -g gene_id \
-a Homo_sapiens.GRCh38.98.gtf \
-o all_counts.txt \
*.sort.bam

`all_counts.txt`をローカルPCにダウンロードする